In [1]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [2]:
NUM_ROWS = 4
NUM_COLUMNS = 3
STEP_COST = -0.04
GAMMA = 0.95

In [3]:
W = [ (2, 1) ]
T = [ (2, 1), (0, 1), (0, 2) ]

In [4]:
def is_valid(
    state: (int, int)
) -> bool:

    row = state[0]
    col = state[1]

    if not 0 <= row < NUM_ROWS:
        return False

    if not 0 <= col < NUM_COLUMNS:
        return False

    if state in W:
        return False

    return True

In [5]:
def generate_actions(
    row: int,
    col: int,
    p: float
) -> dict[str, dict[(int, int), float]]:

    acc = p
    fail = (1 - p) / 2

    actions = {}
    actions['up'] = { (row - 1, col): acc, (row, col - 1): fail, (row, col + 1): fail }
    actions['down'] = { (row + 1, col): acc, (row, col - 1): fail, (row, col + 1): fail }
    actions['left'] = { (row, col - 1): acc, (row - 1, col): fail, (row + 1, col): fail }
    actions['right'] = { (row, col + 1): acc, (row - 1, col): fail, (row + 1, col): fail }

    return constrain_actions(row, col, actions)

In [6]:
def constrain_actions(
    row: int,
    col: int,
    actions: dict[str, dict[(int, int), float]]
) -> dict[str, dict[(int, int), float]]:

    actions_new = {}
    for action, transition in actions.items():
        residue = 0
        transitions_new = {}
        for state, prob in transition.items():
            if is_valid(state):
                transitions_new[state] = prob
            else:
                residue += prob
        if residue != 0:
            transitions_new[(row, col)] = residue
        actions_new[action] = transitions_new

    return actions_new

In [7]:
def evaluate_actions(
    U: list[list[int]],
    actions: dict[str, dict[(int, int), float]]
) -> dict[str, float]:

    evals = {}
    for action, transition in actions.items():
        value = 0
        for state, prob in transition.items():
            value += prob * U[state[0]][state[1]]
        evals[action] = value

    return evals

In [8]:
def optimal_action(
    evals: dict[str, float]
) -> (str, float):

    opt_value = -1e30
    opt_strategy = None
    for strategy, value in evals.items():
        if value > opt_value:
            opt_value = value
            opt_strategy = strategy

    return opt_strategy, opt_value

In [9]:
def value_iteration(
    p: float
) -> (np.ndarray, np.ndarray):

    U = np.zeros((NUM_ROWS, NUM_COLUMNS))
    P = np.full((NUM_ROWS, NUM_COLUMNS), 'none', dtype='object') 
    U[0][1] = -1
    U[0][2] = 1

    k = 0
    while True:
        U_new = np.copy(U)
        P_new = np.copy(P)
        for i, U_i in enumerate(U):
            for j, U_ij in enumerate(U_i):
                if (i, j) not in T:
                    actions = generate_actions(i, j, p)
                    evals = evaluate_actions(U, actions)
                    action, val = optimal_action(evals)
                    U_new[i][j] = STEP_COST + GAMMA * val
                    P_new[i][j] = action
        k = k + 1
        if (np.absolute(U_new - U) <= 1e-4).all():
            break
        U = U_new
        P = P_new

    return U, P

In [10]:
for i in range(1, 10):
    p = i / 10
    utility, policy = value_iteration(p)
    print('p =', p)
    print(utility)
    print(policy)
    print()

p = 0.1
[[ 0.08 -1.    1.  ]
 [ 0.18  0.32  0.59]
 [ 0.02  0.    0.33]
 [-0.04  0.04  0.22]]
[['left' 'none' 'none']
 ['up' 'down' 'right']
 ['left' 'none' 'left']
 ['up' 'up' 'right']]

p = 0.2
[[-0.02 -1.    1.  ]
 [ 0.08  0.25  0.55]
 [-0.06  0.    0.28]
 [-0.09 -0.02  0.16]]
[['left' 'none' 'none']
 ['up' 'down' 'right']
 ['left' 'none' 'left']
 ['up' 'up' 'right']]

p = 0.3
[[-0.08 -1.    1.  ]
 [ 0.03  0.23  0.58]
 [-0.06  0.    0.38]
 [-0.04  0.07  0.22]]
[['left' 'none' 'none']
 ['down' 'down' 'right']
 ['left' 'none' 'up']
 ['down' 'right' 'right']]

p = 0.4
[[-0.14 -1.    1.  ]
 [-0.02  0.2   0.59]
 [-0.08  0.    0.43]
 [-0.01  0.13  0.25]]
[['left' 'none' 'none']
 ['down' 'down' 'right']
 ['left' 'none' 'up']
 ['down' 'right' 'right']]

p = 0.5
[[-0.1  -1.    1.  ]
 [ 0.05  0.24  0.64]
 [ 0.    0.    0.51]
 [ 0.09  0.23  0.33]]
[['left' 'none' 'none']
 ['right' 'down' 'up']
 ['down' 'none' 'up']
 ['right' 'right' 'up']]

p = 0.6
[[-0.05 -1.    1.  ]
 [ 0.14  0.29  0.72]
 [ 0